From Launch to Legacy: Tracking US Spacecraft Lifecycles

Purpose of the following code is to load from CSV, transform the data for the scope of the topic and saving the data to SQLite

In [14]:
import requests
import pandas as pd
import sqlite3
from datetime import datetime

In [15]:
# Importing CSV into Dataframe
space_mission_df = pd.read_csv('../tin/Resources/space_missions.csv', encoding='latin1')
space_mission_df.columns = space_mission_df.columns.str.lower()
space_mission_df.head()

,company,location,date,time,rocket,mission,rocketstatus,price,missionstatus
0,RVSN USSR,"Site 1/5, Baikonur Cosmodrome, Kazakhstan",1957-10-04,19:28:00,Sputnik 8K71PS,Sputnik-1,Retired,NaN,Success
1,RVSN USSR,"Site 1/5, Baikonur Cosmodrome, Kazakhstan",1957-11-03,02:30:00,Sputnik 8K71PS,Sputnik-2,Retired,NaN,Success
2,US Navy,"LC-18A, Cape Canaveral AFS, Florida, USA",1957-12-06,16:44:00,Vanguard,Vanguard TV3,Retired,NaN,Failure
3,AMBA,"LC-26A, Cape Canaveral AFS, Florida, USA",1958-02-01,03:48:00,Juno I,Explorer 1,Retired,NaN,Success
4,US Navy,"LC-18A, Cape Canaveral AFS, Florida, USA",1958-02-05,07:33:00,Vanguard,Vanguard TV3BU,Retired,NaN,Failure


In [16]:
# Dropping Price Column
space_mission_df = space_mission_df.drop('price', axis='columns')
space_mission_df.head()

,company,location,date,time,rocket,mission,rocketstatus,missionstatus
0,RVSN USSR,"Site 1/5, Baikonur Cosmodrome, Kazakhstan",1957-10-04,19:28:00,Sputnik 8K71PS,Sputnik-1,Retired,Success
1,RVSN USSR,"Site 1/5, Baikonur Cosmodrome, Kazakhstan",1957-11-03,02:30:00,Sputnik 8K71PS,Sputnik-2,Retired,Success
2,US Navy,"LC-18A, Cape Canaveral AFS, Florida, USA",1957-12-06,16:44:00,Vanguard,Vanguard TV3,Retired,Failure
3,AMBA,"LC-26A, Cape Canaveral AFS, Florida, USA",1958-02-01,03:48:00,Juno I,Explorer 1,Retired,Success
4,US Navy,"LC-18A, Cape Canaveral AFS, Florida, USA",1958-02-05,07:33:00,Vanguard,Vanguard TV3BU,Retired,Failure


In [17]:
# Data import into SQLite
conn = sqlite3.connect('space_data.db')
space_mission_df.to_sql('missions', conn, if_exists='replace')

4630

In [18]:
# Identifying all Companies in table

conn = sqlite3.connect('space_data.db')
cursor = conn.cursor()

cursor.execute("SELECT DISTINCT company FROM missions")
unique_companies = cursor.fetchall()
print("Unique Companies:")
for company in unique_companies:
    print(company[0])
print(f"Total Companies: {len(unique_companies)}")


Unique Companies:
RVSN USSR
US Navy
AMBA
US Air Force
NASA
General Dynamics
Martin Marietta
Armée de l'Air
UT
ASI
OKB-586
CECLES
Yuzhmash
CASC
CNES
Roscosmos
ISAS
RAE
MHI
ISRO
ESA
Arianespace
IAI
Boeing
Douglas
Northrop
Lockheed
VKS RF
MITT
ILS
EER
AEB
SRC
KCST
Sea Launch
Kosmotras
Eurockot
CASIC
Starsem
SpaceX
ULA
Land Launch
ISA
KARI
JAXA
Khrunichev
Blue Origin
Sandia
ExPace
Rocket Lab
Exos
Landspace
OneSpace
i-Space
IRGC
Virgin Orbit
Astra
Galactic Energy
Virgin Galactic
GK LS
Firefly
CAS Space
Total Companies: 62


In [19]:
# US Govt and Private Sector
us_companies = [
    "NASA", "US Navy", "US Air Force", "SpaceX", "Blue Origin", "Boeing",
    "Northrop", "Lockheed", "ULA", "General Dynamics", "Martin Marietta"
]

query = f"SELECT * FROM missions WHERE company IN ({','.join(['?']*len(us_companies))})"
cursor.execute(query, us_companies)
us_only = cursor.fetchall()

usa_df = pd.DataFrame(us_only, columns=[desc[0] for desc in cursor.description])
usa_df = usa_df.drop('index', axis='columns')
usa_df


,company,location,date,time,rocket,mission,rocketstatus,missionstatus
0,US Navy,"LC-18A, Cape Canaveral AFS, Florida, USA",1957-12-06,16:44:00,Vanguard,Vanguard TV3,Retired,Failure
1,US Navy,"LC-18A, Cape Canaveral AFS, Florida, USA",1958-02-05,07:33:00,Vanguard,Vanguard TV3BU,Retired,Failure
2,US Navy,"LC-18A, Cape Canaveral AFS, Florida, USA",1958-03-17,12:15:00,Vanguard,Vanguard 1,Retired,Success
3,US Navy,"LC-18A, Cape Canaveral AFS, Florida, USA",1958-04-28,02:53:00,Vanguard,Vanguard TV5,Retired,Failure
4,US Navy,"LC-18A, Cape Canaveral AFS, Florida, USA",1958-05-28,03:46:00,Vanguard,Vanguard SLV-1,Retired,Failure
...,...,...,...,...,...,...,...,...
1399,SpaceX,"SLC-4E, Vandenberg SFB, California, USA",2022-07-11,01:39:00,Falcon 9 Block 5,Starlink Group 3-1,Active,Success
1400,SpaceX,"LC-39A, Kennedy Space Center, Florida, USA",2022-07-15,00:44:00,Falcon 9 Block 5,CRS SpX-25,Active,Success
1401,SpaceX,"SLC-40, Cape Canaveral SFS, Florida, USA",2022-07-17,14:20:00,Falcon 9 Block 5,Starlink Group 4-22,Active,Success
1402,SpaceX,"SLC-4E, Vandenberg SFB, California, USA",2022-07-22,17:39:00,Falcon 9 Block 5,Starlink Group 3-2,Active,Success


In [20]:
us_unique_rockets = usa_df['rocket'].unique()
us_unique_rockets

array(['Vanguard', 'NOTS-EV-1 Pilot II', 'Thor-DM 18 Able I',
       'SM-65B Atlas', 'Thor-DM18 Agena-A', 'Thor DM-18 Able-III',
       'Atlas-D Mercury', 'Thor DM-18 Able-II', 'Atlas-D Able',
       'Atlas-LV3 Agena-A', 'Thor DM-18 Able-IV', 'Thor-DM21 Ablestar',
       'Thor DM-19 Delta', 'Thor DM-21 Agena-B', 'Mercury-Redstone',
       'Scout X-1', 'Atlas-LV3 Agena-B', 'Blue Scout II', 'Scout X-2',
       'Atlas-LV3C Centaur-A', 'Scout X-2M', 'Thor DM-21 Agena-D',
       'Delta A', 'Delta B', 'Scout X-3', 'Scout X-3M', 'Scout X-4',
       'Atlas-LV3 Agena-D', 'Scout X-2B', 'Atlas-LV3C Centaur-B',
       'Saturn I', 'Titan II GLV', 'Atlas-LV3C Centaur-C',
       'Atlas-SLV3 Agena-D', 'Titan IIIA', 'Atlas-D OV1', 'Titan IIIC',
       'Atlas-LV3C Centaur-D', 'Saturn IB', 'Atlas-SLV3B Agena-D',
       'Atlas SLV-3', 'Atlas-SLV3 Agena-B', 'Titan IIIB',
       'Atlas-SLV3C Centaur-D', 'Saturn V', 'Atlas-SLV3A Agena-D',
       'Atlas-E/F OV1', 'Atlas-SLV3 Burner-2', 'Titan III(23)B',
     

In [21]:
# US Govt. and Private Sector operation days data

# Convert 'date' column to datetime format if not already
usa_df["date"] = pd.to_datetime(usa_df["date"])

# Group by Company & Rocket and calculate first & last mission dates
us_rocket_lifetimes = usa_df.groupby(["company", "rocket"]).agg(
    first_mission=("date", "min"),
    last_mission=("date", "max")
).reset_index()

# Calculate days in operation
us_rocket_lifetimes["operation_days"] = (us_rocket_lifetimes["last_mission"] - us_rocket_lifetimes["first_mission"]).dt.days

# Count total missions per company + rocket
us_missions_per_rocket = usa_df.groupby(["company", "rocket"]).size().reset_index(name="total_missions")

# Merge mission count with rocket lifetimes dataset
us_rocket_lifetimes = us_rocket_lifetimes.merge(us_missions_per_rocket, on=["company", "rocket"], how="left")

# Determine Rocket Status (Active vs. Retired)
active_threshold = datetime.now() - pd.DateOffset(years=5)

us_rocket_lifetimes["rocketstatus"] = us_rocket_lifetimes["last_mission"].apply(
    lambda x: "Active" if x >= active_threshold else "Retired"
)

# Sort by total missions (highest first)
us_rocket_lifetimes = us_rocket_lifetimes.sort_values(by="total_missions", ascending=False)

# Display results
us_rocket_lifetimes.head(20)

,company,rocket,first_mission,last_mission,operation_days,total_missions,rocketstatus
126,SpaceX,Falcon 9 Block 5,2018-05-11,2022-07-24,1535,111,Active
17,Boeing,Delta II 7925,1990-11-26,2006-11-17,5835,56,Retired
53,General Dynamics,Atlas-SLV3 Agena-D,1964-08-14,1967-11-05,1178,47,Retired
106,NASA,Space Shuttle Discovery,1984-08-30,2011-02-24,9674,39,Retired
165,US Air Force,Thor DM-21 Agena-B,1960-10-26,1962-11-24,759,38,Retired
131,ULA,Atlas V 401,2007-03-09,2021-10-16,5335,36,Active
103,NASA,Space Shuttle Atlantis,1985-10-03,2011-07-08,9409,33,Retired
120,Northrop,Pegasus XL,1994-06-27,2021-06-13,9848,30,Active
105,NASA,Space Shuttle Columbia,1981-04-12,2003-01-16,7949,28,Retired
46,General Dynamics,Atlas-LV3 Agena-B,1961-07-12,1965-03-21,1348,28,Retired


In [22]:
# Define reusability mapping
reusability_mapping = {
    # Reusable Rockets
    "Space Shuttle Columbia": "Reusable",
    "Space Shuttle Challenger": "Reusable",
    "Space Shuttle Discovery": "Reusable",
    "Space Shuttle Atlantis": "Reusable",
    "Space Shuttle Endeavour": "Reusable",
    "Falcon 9 v1.0": "Reusable",
    "Falcon 9 v1.1": "Reusable",
    "Falcon 9 Block 3": "Reusable",
    "Falcon 9 Block 4": "Reusable",
    "Falcon 9 Block 5": "Reusable",
    "Falcon Heavy": "Reusable",
    "New Shepard": "Reusable",
    "Starship Prototype": "Reusable",

    # Expendable Rockets
    "Vanguard": "Expendable",
    "Mercury-Redstone": "Expendable",
    "Atlas-D Mercury": "Expendable",
    "Thor DM-18 Able": "Expendable",
    "Titan II GLV": "Expendable",
    "Saturn I": "Expendable",
    "Saturn IB": "Expendable",
    "Saturn V": "Expendable",
    "Delta II 7925": "Expendable",
    "Delta IV Heavy": "Expendable",
    "Delta III 8930": "Expendable",
    "Atlas V 401": "Expendable",
    "Atlas V 421": "Expendable",
    "Atlas V 411": "Expendable",
    "Atlas V 431": "Expendable",
    "Atlas V 501": "Expendable",
    "Atlas V 531": "Expendable",
    "Atlas V 541": "Expendable",
    "Atlas V 551": "Expendable",
    "Antares 110": "Expendable",
    "Antares 120": "Expendable",
    "Antares 130": "Expendable",
    "Antares 230": "Expendable",
    "Antares 230+": "Expendable",
    "Ares 1-X": "Expendable"
}

# Apply mapping to the DataFrame
us_rocket_lifetimes["reusability"] = us_rocket_lifetimes["rocket"].map(reusability_mapping).fillna("Unknown")

# Display results
us_rocket_lifetimes.head(20)

,company,rocket,first_mission,last_mission,operation_days,total_missions,rocketstatus,reusability
126,SpaceX,Falcon 9 Block 5,2018-05-11,2022-07-24,1535,111,Active,Reusable
17,Boeing,Delta II 7925,1990-11-26,2006-11-17,5835,56,Retired,Expendable
53,General Dynamics,Atlas-SLV3 Agena-D,1964-08-14,1967-11-05,1178,47,Retired,Unknown
106,NASA,Space Shuttle Discovery,1984-08-30,2011-02-24,9674,39,Retired,Reusable
165,US Air Force,Thor DM-21 Agena-B,1960-10-26,1962-11-24,759,38,Retired,Unknown
131,ULA,Atlas V 401,2007-03-09,2021-10-16,5335,36,Active,Expendable
103,NASA,Space Shuttle Atlantis,1985-10-03,2011-07-08,9409,33,Retired,Reusable
120,Northrop,Pegasus XL,1994-06-27,2021-06-13,9848,30,Active,Unknown
105,NASA,Space Shuttle Columbia,1981-04-12,2003-01-16,7949,28,Retired,Reusable
46,General Dynamics,Atlas-LV3 Agena-B,1961-07-12,1965-03-21,1348,28,Retired,Unknown


In [23]:
# Clearing out rows without reusability data
us_rocket_lifetimes = us_rocket_lifetimes[us_rocket_lifetimes["reusability"] != "Unknown"]
us_rocket_lifetimes

,company,rocket,first_mission,last_mission,operation_days,total_missions,rocketstatus,reusability
126,SpaceX,Falcon 9 Block 5,2018-05-11,2022-07-24,1535,111,Active,Reusable
17,Boeing,Delta II 7925,1990-11-26,2006-11-17,5835,56,Retired,Expendable
106,NASA,Space Shuttle Discovery,1984-08-30,2011-02-24,9674,39,Retired,Reusable
131,ULA,Atlas V 401,2007-03-09,2021-10-16,5335,36,Active,Expendable
103,NASA,Space Shuttle Atlantis,1985-10-03,2011-07-08,9409,33,Retired,Reusable
105,NASA,Space Shuttle Columbia,1981-04-12,2003-01-16,7949,28,Retired,Reusable
124,SpaceX,Falcon 9 Block 3,2015-12-22,2018-02-22,793,25,Retired,Reusable
107,NASA,Space Shuttle Endeavour,1992-05-07,2011-05-16,6948,25,Retired,Reusable
0,Blue Origin,New Shepard,2015-04-29,2022-06-04,2593,21,Active,Reusable
128,SpaceX,Falcon 9 v1.1,2013-09-29,2016-01-17,840,15,Retired,Reusable


In [24]:
# Save to SQLite
us_rocket_lifetimes.to_sql("us_operation_time", conn, if_exists="replace", index=False)


39